In [1]:
import numpy as np
import pandas as pd
# import datetime as dt
# import seaborn as sns
# import urllib.request
# from io import StringIO
# import requests
#import missingno as msno

# matplotlib
# from matplotlib import pyplot as plt
# from matplotlib.dates import date2num, num2date
# from matplotlib import dates as mdates
# from matplotlib import ticker
# from matplotlib.colors import ListedColormap
# from matplotlib.patches import Patch

# scipy specifics
# from scipy import stats as sps
# from scipy.interpolate import interp1d

In [2]:
try:
    from urllib.request import Request, urlopen  # Python 3
except ImportError:
    from urllib2 import Request, urlopen  # Python 2

req = Request('https://files.minsa.gob.pe/s/eRqxR35ZCxrzNgr/download')
req.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0')
content = urlopen(req)

test = pd.read_csv(content, sep=";", na_values='EN INVESTIGACIÓN', encoding='utf-8')
test

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20221203,LIMA,LIMA,EL AGUSTINO,PR,51.0,MASCULINO,20200516.0,150111.0,13865972.0
1,20221203,AREQUIPA,AREQUIPA,AREQUIPA,PCR,34.0,MASCULINO,20220707.0,40101.0,13866130.0
2,20221203,LIMA,LIMA,SANTIAGO DE SURCO,AG,35.0,MASCULINO,20221129.0,150140.0,13866165.0
3,20221203,LIMA,LIMA,JESUS MARIA,AG,20.0,FEMENINO,20220110.0,150113.0,13866369.0
4,20221203,LIMA,LIMA,SAN JUAN DE LURIGANCHO,AG,24.0,MASCULINO,20221114.0,150132.0,13866398.0
...,...,...,...,...,...,...,...,...,...,...
4300569,20221203,LIMA,LIMA,COMAS,PR,26.0,MASCULINO,20200525.0,150110.0,NaN
4300570,20221203,LORETO,MAYNAS,IQUITOS,AG,64.0,MASCULINO,20220630.0,160101.0,NaN
4300571,20221203,CALLAO,CALLAO,CALLAO,PCR,39.0,MASCULINO,20221203.0,70101.0,NaN
4300572,20221203,LIMA,LIMA,COMAS,PCR,46.0,MASCULINO,20210218.0,150110.0,NaN


In [3]:
#content= "positivos_covid.csv"

#test = pd.read_csv(content, sep=";", na_values='EN INVESTIGACIÓN', encoding='utf-8')
#test

In [4]:
test['FECHA_RESULTADO'] = pd.to_datetime(test['FECHA_RESULTADO'].astype(str), format='%Y%m%d')
test.tail()

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
4300569,20221203,LIMA,LIMA,COMAS,PR,26.0,MASCULINO,2020-05-25,150110.0,NaN
4300570,20221203,LORETO,MAYNAS,IQUITOS,AG,64.0,MASCULINO,2022-06-30,160101.0,NaN
4300571,20221203,CALLAO,CALLAO,CALLAO,PCR,39.0,MASCULINO,2022-12-03,70101.0,NaN
4300572,20221203,LIMA,LIMA,COMAS,PCR,46.0,MASCULINO,2021-02-18,150110.0,NaN
4300573,20221203,LIMA,LIMA,RIMAC,PCR,23.0,FEMENINO,2020-12-22,150128.0,NaN


In [5]:
test.isnull().sum()

FECHA_CORTE             0
DEPARTAMENTO            0
PROVINCIA          215547
DISTRITO           215547
METODODX                0
EDAD                  353
SEXO                    1
FECHA_RESULTADO      2023
UBIGEO             215547
id_persona          78964
dtype: int64

In [6]:
indice_departamento = pd.read_csv('https://raw.githubusercontent.com/annaabsi/git-scraper-covid19/main/resultados/positivos_por_departamentos.csv')
indice_departamento

,DEPARTAMENTO,METODODX,POBLACION,INDICE
0,AMAZONAS,50224,452125,11108
1,ANCASH,152345,1189403,12809
2,APURIMAC,44938,440629,10199
3,AREQUIPA,273875,1488247,18403
4,AYACUCHO,54445,658081,8273
5,CAJAMARCA,109642,1528904,7171
6,CALLAO,165506,1090990,15170
7,CUSCO,139544,1392648,10020
8,HUANCAVELICA,29255,414882,7051
9,HUANUCO,59018,823560,7166


In [7]:
test = pd.merge(test, indice_departamento,
                        how="left", on=["DEPARTAMENTO"])
test

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
0,20221203,LIMA,LIMA,EL AGUSTINO,PR,51.0,MASCULINO,2020-05-16,150111.0,13865972.0,2013848,10741923,18748
1,20221203,AREQUIPA,AREQUIPA,AREQUIPA,PCR,34.0,MASCULINO,2022-07-07,40101.0,13866130.0,273875,1488247,18403
2,20221203,LIMA,LIMA,SANTIAGO DE SURCO,AG,35.0,MASCULINO,2022-11-29,150140.0,13866165.0,2013848,10741923,18748
3,20221203,LIMA,LIMA,JESUS MARIA,AG,20.0,FEMENINO,2022-01-10,150113.0,13866369.0,2013848,10741923,18748
4,20221203,LIMA,LIMA,SAN JUAN DE LURIGANCHO,AG,24.0,MASCULINO,2022-11-14,150132.0,13866398.0,2013848,10741923,18748
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4300569,20221203,LIMA,LIMA,COMAS,PR,26.0,MASCULINO,2020-05-25,150110.0,NaN,2013848,10741923,18748
4300570,20221203,LORETO,MAYNAS,IQUITOS,AG,64.0,MASCULINO,2022-06-30,160101.0,NaN,63642,1096407,5805
4300571,20221203,CALLAO,CALLAO,CALLAO,PCR,39.0,MASCULINO,2022-12-03,70101.0,NaN,165506,1090990,15170
4300572,20221203,LIMA,LIMA,COMAS,PCR,46.0,MASCULINO,2021-02-18,150110.0,NaN,2013848,10741923,18748


In [8]:
poblacion_csv = pd.read_csv('poblacion_provincia.csv')
poblacion_csv

,UBIGEO,PROVINCIA,POBLACION
0,10100,CHACHAPOYAS,63188
1,10200,BAGUA,84672
2,10300,BONGARA,26830
3,10400,CONDORCANQUI,51344
4,10500,LUYA,47827
...,...,...,...
191,240300,ZARUMILLA,56038
192,250100,CORONEL PORTILLO,447733
193,250200,ATALAYA,61049
194,250300,PADRE ABAD,77044


In [9]:
poblacion_dict = poblacion_csv.to_dict('split')
poblacion_dict['data']

[[10100, 'CHACHAPOYAS', 63188],
 [10200, 'BAGUA', 84672],
 [10300, 'BONGARA', 26830],
 [10400, 'CONDORCANQUI', 51344],
 [10500, 'LUYA', 47827],
 [10600, 'RODRIGUEZ DE MENDOZA', 33651],
 [10700, 'UTCUBAMBA', 119294],
 [20100, 'HUARAZ', 185276],
 [20200, 'AIJA', 6433],
 [20300, 'ANTONIO RAYMONDI', 13950],
 [20400, 'ASUNCION', 7710],
 [20500, 'BOLOGNESI', 24012],
 [20600, 'CARHUAZ', 50007],
 [20700, 'CARLOS FERMIN FITZCARRALD', 18496],
 [20800, 'CASMA', 57256],
 [20900, 'CORONGO', 8017],
 [21000, 'HUARI', 63264],
 [21100, 'HUARMEY', 33066],
 [21200, 'HUAYLAS', 56557],
 [21300, 'MARISCAL LUZURIAGA', 21787],
 [21400, 'OCROS', 7224],
 [21500, 'PALLASCA', 24371],
 [21600, 'POMABAMBA', 26675],
 [21700, 'RECUAY', 18085],
 [21800, 'SANTA', 474053],
 [21900, 'SIHUAS', 28630],
 [22000, 'YUNGAY', 55769],
 [30100, 'ABANCAY', 120116],
 [30200, 'ANDAHUAYLAS', 150758],
 [30300, 'ANTABAMBA', 11781],
 [30400, 'AYMARAES', 24570],
 [30500, 'COTABAMBAS', 55208],
 [30600, 'CHINCHEROS', 46544],
 [30700, 'GRAU

In [10]:
poblacion =[]
for x in range(len(poblacion_dict['data'])):
    array = poblacion_dict['data'][x][2]
    poblacion.append(array)
    
poblacion

[63188,
 84672,
 26830,
 51344,
 47827,
 33651,
 119294,
 185276,
 6433,
 13950,
 7710,
 24012,
 50007,
 18496,
 57256,
 8017,
 63264,
 33066,
 56557,
 21787,
 7224,
 24371,
 26675,
 18085,
 474053,
 28630,
 55769,
 120116,
 150758,
 11781,
 24570,
 55208,
 46544,
 21759,
 1175765,
 61708,
 43690,
 34743,
 97458,
 16426,
 54851,
 12797,
 317801,
 32482,
 8341,
 97205,
 75277,
 51838,
 29139,
 9909,
 9292,
 19866,
 17063,
 388170,
 83167,
 83916,
 151714,
 29357,
 123948,
 83913,
 203724,
 145770,
 51678,
 47114,
 22638,
 38602,
 1129854,
 511019,
 24000,
 63131,
 71582,
 34754,
 106476,
 70143,
 62059,
 167910,
 26644,
 47579,
 101735,
 70043,
 121265,
 37503,
 53901,
 14588,
 33883,
 18182,
 85995,
 315799,
 53247,
 32427,
 16372,
 52095,
 138275,
 29160,
 50086,
 36987,
 17114,
 18705,
 445752,
 262110,
 78472,
 14832,
 174016,
 595183,
 59138,
 167385,
 88405,
 22757,
 239105,
 91849,
 40041,
 57604,
 1118724,
 123480,
 15982,
 86411,
 30987,
 85091,
 112970,
 85092,
 168670,
 55868

In [11]:
provincia =[]
for x in range(len(poblacion_dict['data'])):
    array = poblacion_dict['data'][x][1]
    provincia.append(array)
    
provincia

['CHACHAPOYAS',
 'BAGUA',
 'BONGARA',
 'CONDORCANQUI',
 'LUYA',
 'RODRIGUEZ DE MENDOZA',
 'UTCUBAMBA',
 'HUARAZ',
 'AIJA',
 'ANTONIO RAYMONDI',
 'ASUNCION',
 'BOLOGNESI',
 'CARHUAZ',
 'CARLOS FERMIN FITZCARRALD',
 'CASMA',
 'CORONGO',
 'HUARI',
 'HUARMEY',
 'HUAYLAS',
 'MARISCAL LUZURIAGA',
 'OCROS',
 'PALLASCA',
 'POMABAMBA',
 'RECUAY',
 'SANTA',
 'SIHUAS',
 'YUNGAY',
 'ABANCAY',
 'ANDAHUAYLAS',
 'ANTABAMBA',
 'AYMARAES',
 'COTABAMBAS',
 'CHINCHEROS',
 'GRAU',
 'AREQUIPA',
 'CAMANA',
 'CARAVELI',
 'CASTILLA',
 'CAYLLOMA',
 'CONDESUYOS',
 'ISLAY',
 'LA UNION',
 'HUAMANGA',
 'CANGALLO',
 'HUANCA SANCOS',
 'HUANTA',
 'LA MAR',
 'LUCANAS',
 'PARINACOCHAS',
 'PAUCAR DEL SARA SARA',
 'SUCRE',
 'VICTOR FAJARDO',
 'VILCAS HUAMAN',
 'CAJAMARCA',
 'CAJABAMBA',
 'CELENDIN',
 'CHOTA',
 'CONTUMAZA',
 'CUTERVO',
 'HUALGAYOC',
 'JAEN',
 'SAN IGNACIO',
 'SAN MARCOS',
 'SAN MIGUEL',
 'SAN PABLO',
 'SANTA CRUZ',
 'CALLAO',
 'CUSCO',
 'ACOMAYO',
 'ANTA',
 'CALCA',
 'CANAS',
 'CANCHIS',
 'CHUMBIVILCAS',


In [12]:
res = {provincia[i]: poblacion[i] for i in range(len(provincia))}
res

{'CHACHAPOYAS': 63188,
 'BAGUA': 84672,
 'BONGARA': 26830,
 'CONDORCANQUI': 51344,
 'LUYA': 47827,
 'RODRIGUEZ DE MENDOZA': 33651,
 'UTCUBAMBA': 119294,
 'HUARAZ': 185276,
 'AIJA': 6433,
 'ANTONIO RAYMONDI': 13950,
 'ASUNCION': 7710,
 'BOLOGNESI': 24012,
 'CARHUAZ': 50007,
 'CARLOS FERMIN FITZCARRALD': 18496,
 'CASMA': 57256,
 'CORONGO': 8017,
 'HUARI': 63264,
 'HUARMEY': 33066,
 'HUAYLAS': 56557,
 'MARISCAL LUZURIAGA': 21787,
 'OCROS': 7224,
 'PALLASCA': 24371,
 'POMABAMBA': 26675,
 'RECUAY': 18085,
 'SANTA': 474053,
 'SIHUAS': 28630,
 'YUNGAY': 55769,
 'ABANCAY': 120116,
 'ANDAHUAYLAS': 150758,
 'ANTABAMBA': 11781,
 'AYMARAES': 24570,
 'COTABAMBAS': 55208,
 'CHINCHEROS': 46544,
 'GRAU': 21759,
 'AREQUIPA': 1175765,
 'CAMANA': 61708,
 'CARAVELI': 43690,
 'CASTILLA': 34743,
 'CAYLLOMA': 97458,
 'CONDESUYOS': 16426,
 'ISLAY': 54851,
 'LA UNION': 12797,
 'HUAMANGA': 317801,
 'CANGALLO': 32482,
 'HUANCA SANCOS': 8341,
 'HUANTA': 97205,
 'LA MAR': 75277,
 'LUCANAS': 51838,
 'PARINACOCHAS':

In [13]:
test['POBLACION'] = test['PROVINCIA'].map(res)

In [14]:
null_columns = test.columns[test.isnull().any()]
test[test["SEXO"].isnull()][null_columns]

,PROVINCIA,DISTRITO,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,POBLACION
3626210,LIMA,LIMA,0.0,NaN,2021-03-13,150101.0,10881464.0,9674755.0


In [15]:
#find_provincia = test['PROVINCIA'] == 'UCAYALI'
find_distrito = test['DEPARTAMENTO'] == 'LORETO'
find_fecha = test['FECHA_RESULTADO'] == '2022-01-02'

data_exploratoria = test[find_distrito & find_fecha]
data_exploratoria

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
757940,20221203,LORETO,MAYNAS,SAN JUAN BAUTISTA,PR,32.0,FEMENINO,2022-01-02,160113.0,20925980.0,63642,550551.0,5805
933933,20221203,LORETO,MAYNAS,PUNCHANA,PR,31.0,MASCULINO,2022-01-02,160108.0,22425039.0,63642,550551.0,5805
1970209,20221203,LORETO,MAYNAS,IQUITOS,AG,24.0,MASCULINO,2022-01-02,160101.0,34702609.0,63642,550551.0,5805
2136813,20221203,LORETO,MAYNAS,PUNCHANA,AG,27.0,MASCULINO,2022-01-02,160108.0,19990922.0,63642,550551.0,5805
2608178,20221203,LORETO,MAYNAS,IQUITOS,AG,69.0,FEMENINO,2022-01-02,160101.0,3428514.0,63642,550551.0,5805


In [16]:
data_exploratoria.groupby("SEXO").count()

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
SEXO,,,,,,,,,,,,
FEMENINO,2,2,2,2,2,2,2,2,2,2,2,2
MASCULINO,3,3,3,3,3,3,3,3,3,3,3,3


In [17]:
test['SEXO'] = test['SEXO'].fillna('FEMENINO')

In [18]:
null_columns = test.columns[test.isnull().any()]
test[test["PROVINCIA"].isnull()][null_columns]

,PROVINCIA,DISTRITO,EDAD,FECHA_RESULTADO,UBIGEO,id_persona,POBLACION
7,NaN,NaN,39.0,2021-06-23,NaN,13866581.0,NaN
78,NaN,NaN,13.0,2022-12-01,NaN,13878231.0,NaN
149,NaN,NaN,68.0,2021-03-01,NaN,13925493.0,NaN
186,NaN,NaN,52.0,2021-07-06,NaN,13877907.0,NaN
287,NaN,NaN,20.0,2022-11-22,NaN,13937412.0,NaN
...,...,...,...,...,...,...,...
4300220,NaN,NaN,8.0,2022-01-21,NaN,NaN,NaN
4300286,NaN,NaN,26.0,2022-01-15,NaN,NaN,NaN
4300487,NaN,NaN,31.0,2022-01-20,NaN,NaN,NaN
4300506,NaN,NaN,0.0,2020-07-03,NaN,NaN,NaN


In [19]:
test['FECHA_RESULTADO'] = pd.to_datetime(test['FECHA_RESULTADO'], errors='coerce', dayfirst=True)
test['FECHA_RESULTADO']

0         2020-05-16
1         2022-07-07
2         2022-11-29
3         2022-01-10
4         2022-11-14
             ...    
4300569   2020-05-25
4300570   2022-06-30
4300571   2022-12-03
4300572   2021-02-18
4300573   2020-12-22
Name: FECHA_RESULTADO, Length: 4300574, dtype: datetime64[ns]

In [20]:
test['POBLACION_CIENMIL'] = test['POBLACION']/100000

#test = test.drop(labels="POBLACION_CIENMIL", axis=1)
test

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
0,20221203,LIMA,LIMA,EL AGUSTINO,PR,51.0,MASCULINO,2020-05-16,150111.0,13865972.0,2013848,9674755.0,18748,96.74755
1,20221203,AREQUIPA,AREQUIPA,AREQUIPA,PCR,34.0,MASCULINO,2022-07-07,40101.0,13866130.0,273875,1175765.0,18403,11.75765
2,20221203,LIMA,LIMA,SANTIAGO DE SURCO,AG,35.0,MASCULINO,2022-11-29,150140.0,13866165.0,2013848,9674755.0,18748,96.74755
3,20221203,LIMA,LIMA,JESUS MARIA,AG,20.0,FEMENINO,2022-01-10,150113.0,13866369.0,2013848,9674755.0,18748,96.74755
4,20221203,LIMA,LIMA,SAN JUAN DE LURIGANCHO,AG,24.0,MASCULINO,2022-11-14,150132.0,13866398.0,2013848,9674755.0,18748,96.74755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4300569,20221203,LIMA,LIMA,COMAS,PR,26.0,MASCULINO,2020-05-25,150110.0,NaN,2013848,9674755.0,18748,96.74755
4300570,20221203,LORETO,MAYNAS,IQUITOS,AG,64.0,MASCULINO,2022-06-30,160101.0,NaN,63642,550551.0,5805,5.50551
4300571,20221203,CALLAO,CALLAO,CALLAO,PCR,39.0,MASCULINO,2022-12-03,70101.0,NaN,165506,1129854.0,15170,11.29854
4300572,20221203,LIMA,LIMA,COMAS,PCR,46.0,MASCULINO,2021-02-18,150110.0,NaN,2013848,9674755.0,18748,96.74755


In [21]:
salidasxsemanas = test.sort_values(by = 'FECHA_RESULTADO')
start_date = "2022-10-12"
end_date = "2022-11-30"

after_start_date = salidasxsemanas["FECHA_RESULTADO"] >= start_date
before_end_date = salidasxsemanas["FECHA_RESULTADO"] <= end_date
between_two_dates = after_start_date & before_end_date

filtered_dates = salidasxsemanas.loc[between_two_dates]

filtered_dates

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
242859,20221203,PUNO,SAN ROMAN,JULIACA,PCR,15.0,FEMENINO,2022-10-12,211101.0,15930301.0,75510,344030.0,6182,3.44030
1421259,20221203,ANCASH,CASMA,CASMA,AG,50.0,FEMENINO,2022-10-12,20801.0,27376820.0,152345,57256.0,12809,0.57256
107492,20221203,TACNA,TACNA,TACNA,AG,27.0,MASCULINO,2022-10-12,230101.0,14812413.0,61849,346192.0,16909,3.46192
3645611,20221203,LA LIBERTAD,VIRU,GUADALUPITO,AG,39.0,FEMENINO,2022-10-12,131203.0,10982238.0,171796,105206.0,8783,1.05206
3807087,20221203,LIMA,LIMA,SAN MIGUEL,PCR,69.0,FEMENINO,2022-10-12,150136.0,12036083.0,2013848,9674755.0,18748,96.74755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2076653,20221203,LIMA,NaN,NaN,AG,47.0,FEMENINO,2022-11-30,NaN,19894962.0,2013848,NaN,18748,NaN
460486,20221203,CALLAO,CALLAO,BELLAVISTA,AG,27.0,FEMENINO,2022-11-30,70102.0,17720044.0,165506,1129854.0,15170,11.29854
1581714,20221203,LIMA,LIMA,CARABAYLLO,AG,31.0,FEMENINO,2022-11-30,150106.0,29346819.0,2013848,9674755.0,18748,96.74755
815464,20221203,LIMA,LIMA,SAN BORJA,AG,31.0,FEMENINO,2022-11-30,150130.0,21398534.0,2013848,9674755.0,18748,96.74755


In [22]:
filtered_dates['PROVINCIA'] = filtered_dates['PROVINCIA'].replace('EN INVESTIGACIÃN', np.nan)
filtered_dates

/tmp/ipykernel_20467/4273863947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dates['PROVINCIA'] = filtered_dates['PROVINCIA'].replace('EN INVESTIGACIÃN', np.nan)


,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
242859,20221203,PUNO,SAN ROMAN,JULIACA,PCR,15.0,FEMENINO,2022-10-12,211101.0,15930301.0,75510,344030.0,6182,3.44030
1421259,20221203,ANCASH,CASMA,CASMA,AG,50.0,FEMENINO,2022-10-12,20801.0,27376820.0,152345,57256.0,12809,0.57256
107492,20221203,TACNA,TACNA,TACNA,AG,27.0,MASCULINO,2022-10-12,230101.0,14812413.0,61849,346192.0,16909,3.46192
3645611,20221203,LA LIBERTAD,VIRU,GUADALUPITO,AG,39.0,FEMENINO,2022-10-12,131203.0,10982238.0,171796,105206.0,8783,1.05206
3807087,20221203,LIMA,LIMA,SAN MIGUEL,PCR,69.0,FEMENINO,2022-10-12,150136.0,12036083.0,2013848,9674755.0,18748,96.74755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2076653,20221203,LIMA,NaN,NaN,AG,47.0,FEMENINO,2022-11-30,NaN,19894962.0,2013848,NaN,18748,NaN
460486,20221203,CALLAO,CALLAO,BELLAVISTA,AG,27.0,FEMENINO,2022-11-30,70102.0,17720044.0,165506,1129854.0,15170,11.29854
1581714,20221203,LIMA,LIMA,CARABAYLLO,AG,31.0,FEMENINO,2022-11-30,150106.0,29346819.0,2013848,9674755.0,18748,96.74755
815464,20221203,LIMA,LIMA,SAN BORJA,AG,31.0,FEMENINO,2022-11-30,150130.0,21398534.0,2013848,9674755.0,18748,96.74755


In [23]:
filtered_dates.isnull().sum()

FECHA_CORTE             0
DEPARTAMENTO            0
PROVINCIA            9821
DISTRITO             9821
METODODX_x              0
EDAD                    1
SEXO                    0
FECHA_RESULTADO         0
UBIGEO               9821
id_persona           2295
METODODX_y              0
POBLACION            9998
INDICE                  0
POBLACION_CIENMIL    9998
dtype: int64

In [24]:
weekly_sales = filtered_dates.groupby(["SEXO","DEPARTAMENTO","PROVINCIA", "POBLACION", "POBLACION_CIENMIL", "INDICE", pd.Grouper(key="FECHA_RESULTADO",freq="W-SUN")]).size()
weekly_sales = weekly_sales.unstack(0).fillna(0)
weekly_sales.sort_values(by=['FECHA_RESULTADO'], inplace=True, ascending=True)
weekly_sales.loc[:,'TOTAL'] = weekly_sales.sum(numeric_only=True, axis=1)

weekly_sales = weekly_sales.reset_index()
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,ANCASH,SANTA,474053.0,4.74053,12809,2022-10-16,11.0,10.0,21.0
1,MOQUEGUA,MARISCAL NIETO,95551.0,0.95551,31738,2022-10-16,2.0,1.0,3.0
2,HUANUCO,HUACAYBAMBA,16372.0,0.16372,7166,2022-10-16,1.0,0.0,1.0
3,LA LIBERTAD,VIRU,105206.0,1.05206,8783,2022-10-16,1.0,0.0,1.0
4,PASCO,OXAPAMPA,100561.0,1.00561,9770,2022-10-16,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...
945,MADRE DE DIOS,MANU,21118.0,0.21118,12129,2022-12-04,1.0,1.0,2.0
946,HUANCAVELICA,HUAYTARA,18182.0,0.18182,7051,2022-12-04,7.0,4.0,11.0
947,LORETO,UCAYALI,62477.0,0.62477,5805,2022-12-04,2.0,0.0,2.0
948,APURIMAC,GRAU,21759.0,0.21759,10199,2022-12-04,1.0,1.0,2.0


In [25]:
weekly_sales.groupby("FECHA_RESULTADO").count()

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FEMENINO,MASCULINO,TOTAL
FECHA_RESULTADO,,,,,,,,
2022-10-16,72,72,72,72,72,72,72,72
2022-10-23,86,86,86,86,86,86,86,86
2022-10-30,84,84,84,84,84,84,84,84
2022-11-06,88,88,88,88,88,88,88,88
2022-11-13,111,111,111,111,111,111,111,111
2022-11-20,153,153,153,153,153,153,153,153
2022-11-27,175,175,175,175,175,175,175,175
2022-12-04,181,181,181,181,181,181,181,181


In [26]:
cero_cases = weekly_sales['PROVINCIA']== "ATALAYA"
data_cero = weekly_sales[cero_cases]
data_cero

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
305,UCAYALI,ATALAYA,61049.0,0.61049,7570,2022-11-06,0.0,1.0,1.0
604,UCAYALI,ATALAYA,61049.0,0.61049,7570,2022-11-27,1.0,0.0,1.0
846,UCAYALI,ATALAYA,61049.0,0.61049,7570,2022-12-04,1.0,0.0,1.0


In [27]:
weekly_sales[['FEMENINO','MASCULINO', 'TOTAL']] = weekly_sales[['FEMENINO','MASCULINO', 'TOTAL']].div(weekly_sales['POBLACION_CIENMIL'].values,axis=0)
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,ANCASH,SANTA,474053.0,4.74053,12809,2022-10-16,2.320416,2.109469,4.429884
1,MOQUEGUA,MARISCAL NIETO,95551.0,0.95551,31738,2022-10-16,2.093123,1.046562,3.139685
2,HUANUCO,HUACAYBAMBA,16372.0,0.16372,7166,2022-10-16,6.107989,0.000000,6.107989
3,LA LIBERTAD,VIRU,105206.0,1.05206,8783,2022-10-16,0.950516,0.000000,0.950516
4,PASCO,OXAPAMPA,100561.0,1.00561,9770,2022-10-16,0.000000,2.983264,2.983264
...,...,...,...,...,...,...,...,...,...
945,MADRE DE DIOS,MANU,21118.0,0.21118,12129,2022-12-04,4.735297,4.735297,9.470594
946,HUANCAVELICA,HUAYTARA,18182.0,0.18182,7051,2022-12-04,38.499615,21.999780,60.499395
947,LORETO,UCAYALI,62477.0,0.62477,5805,2022-12-04,3.201178,0.000000,3.201178
948,APURIMAC,GRAU,21759.0,0.21759,10199,2022-12-04,4.595799,4.595799,9.191599


In [28]:
weekly_sales.FEMENINO = weekly_sales.FEMENINO.round()
weekly_sales.MASCULINO = weekly_sales.MASCULINO.round()
weekly_sales.TOTAL = weekly_sales.FEMENINO + weekly_sales.MASCULINO
weekly_sales.FECHA_RESULTADO = weekly_sales.FECHA_RESULTADO.dt.strftime('%Y-%m-%d')
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,ANCASH,SANTA,474053.0,4.74053,12809,2022-10-16,2.0,2.0,4.0
1,MOQUEGUA,MARISCAL NIETO,95551.0,0.95551,31738,2022-10-16,2.0,1.0,3.0
2,HUANUCO,HUACAYBAMBA,16372.0,0.16372,7166,2022-10-16,6.0,0.0,6.0
3,LA LIBERTAD,VIRU,105206.0,1.05206,8783,2022-10-16,1.0,0.0,1.0
4,PASCO,OXAPAMPA,100561.0,1.00561,9770,2022-10-16,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...
945,MADRE DE DIOS,MANU,21118.0,0.21118,12129,2022-12-04,5.0,5.0,10.0
946,HUANCAVELICA,HUAYTARA,18182.0,0.18182,7051,2022-12-04,38.0,22.0,60.0
947,LORETO,UCAYALI,62477.0,0.62477,5805,2022-12-04,3.0,0.0,3.0
948,APURIMAC,GRAU,21759.0,0.21759,10199,2022-12-04,5.0,5.0,10.0


In [29]:
weekly_sales.to_csv('dataset_covid_total.csv' , index=False)

In [30]:
weekly_sales.to_json('dataset_covid_total.json', orient="table")